In [8]:
# Initialize the dataset for Yorùbá with trust_remote_code set to True



TypeError: MasakhaNERDataset.__init__() got an unexpected keyword argument 'trust_remote_code'

In [ ]:
import nltk

# Download necessary NLTK resources for tokenization
nltk.download('punkt')



: 

In [9]:
from pprint import pprint
import datasets

class SumXLDaset:
    def __init__(self, sample_size):
        self.all_data = {}
        self.train_data = {}
        self.valid_data = {}
        self.test_data = {}
        self.sample_size = sample_size
        self.LANGS = [
            "amharic", "arabic", "azerbaijani", "bengali", "burmese",
            "chinese_simplified", "chinese_traditional", "english", "french", 
            "gujarati", "hausa", "hindi", "igbo", "indonesian", "japanese",
            "kirundi", "korean", "kyrgyz", "marathi", "nepali", "oromo", 
            "pashto", "persian", "pidgin", "portuguese", "punjabi", "russian", 
            "scottish_gaelic", "serbian_cyrillic", "serbian_latin", "sinhala", 
            "somali", "spanish", "swahili", "tamil", "telugu", "thai", 
            "tigrinya", "turkish", "ukrainian", "urdu", "uzbek", "vietnamese", 
            "welsh", "yoruba"
        ]
        
        self.load_data()
        
    def load_data(self):
        for lang in self.LANGS:
            try:
                # Load the dataset for each language with trust_remote_code=True
                dataset = datasets.load_dataset('GEM/xlsum', lang, trust_remote_code=True)
                
                # Load samples based on sample_size argument
                if self.sample_size > 0:
                    self.train_data[lang] = dataset['train'].select(range(min(self.sample_size, len(dataset['train']))))
                    self.valid_data[lang] = dataset['validation'].select(range(min(self.sample_size, len(dataset['validation']))))
                    self.test_data[lang] = dataset['test'].select(range(min(self.sample_size, len(dataset['test']))))
                else:
                    self.train_data[lang] = dataset['train']
                    self.valid_data[lang] = dataset['validation']
                    self.test_data[lang] = dataset['test']
                    
                print(f"Loaded data for language: {lang}")
            except Exception as e:
                print(f"Error loading data for language {lang}: {e}")
                
        print(f"Loaded data for all specified languages.")

# Initialize the dataset with a specific sample size (e.g., 2 for testing)
dataset = SumXLDaset(sample_size=2)

# Print the training data for one of the languages, e.g., English
pprint(dataset.train_data['english'])


TypeError: SumXLDaset.__init__() got an unexpected keyword argument 'trust_remote_code'

practice file saving

In [ ]:
# Import necessary libraries
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Sample hypotheses and references for testing
hyps = ["The quick brown fox jumps over the lazy dog", "A journey of a thousand miles begins with a single step"]
refs = ["The quick brown fox leaped over a lazy dog", "A journey of a thousand miles starts with one step"]

# Function to calculate Distinct-1 and Distinct-2
def distinct_n_grams(text, n):
    n_grams = set(zip(*[text[i:] for i in range(n)]))
    return len(n_grams) / len(text) if len(text) > 0 else 0

# Function to evaluate generation metrics
def evaluate_generation_metrics(hyps, refs):
    distinct_1_scores = []
    distinct_2_scores = []
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    # Initialize ROUGE scorer
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Calculate ROUGE and Distinct metrics
    for hyp, ref in zip(hyps, refs):
        rouge_scores = rouge.score(ref, hyp)
        rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
        rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
        rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

        distinct_1 = distinct_n_grams(hyp.split(), 1)
        distinct_2 = distinct_n_grams(hyp.split(), 2)
        distinct_1_scores.append(distinct_1)
        distinct_2_scores.append(distinct_2)

    # Calculate BERTScore
    print("Calculating BERTScore...")
    P, R, F1 = bert_score(hyps, refs, lang='en', rescale_with_baseline=False)
    print("BERTScore calculated.")

    # Organize report dictionary
    report_dict = {
        "ROUGE-1": np.mean(rouge1_scores),
        "ROUGE-2": np.mean(rouge2_scores),
        "ROUGE-L": np.mean(rougeL_scores),
        "BERTScore (P)": np.mean(P),
        "BERTScore (R)": np.mean(R),
        "BERTScore (F1)": np.mean(F1),
        "Distinct-1": np.mean(distinct_1_scores),
        "Distinct-2": np.mean(distinct_2_scores),
        "Ensemble": np.mean([np.mean(rouge1_scores), np.mean(rouge2_scores), np.mean(rougeL_scores), np.mean(F1)])
    }

    return report_dict

# Evaluate and display metrics
print("Evaluating generation metrics...")
report = evaluate_generation_metrics(hyps, refs)
print("Evaluation Report:")
print(json.dumps(report, indent=4))

# Save the evaluation report to JSON
output_path = "evaluation_report_test.json"
with open(output_path, "w") as outfile:
    json.dump(report, outfile, indent=4)

print(f"Report saved successfully to {output_path}")
